In [2]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -l

--2025-11-22 12:23:00--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  23.0MB/s    in 6.6s    

2025-11-22 12:23:08 (19.9 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
total 665408
-rw-r--r-- 1 root root 542497580 Oct  7  2022 games.data
-rwxr-xr-x 1 root root    284672 Oct  1 15:09 golois.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root 138578548 Oct  1 20:02 project2026.zip
drwxr-xr-x 1 root root      4096 Nov 20 14:30 sample_data


# Import

In [3]:
# ================================================================
# 📦 IMPORTS GÉNÉRAUX
# ================================================================
import os, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from google.colab import files  # pour le téléchargement auto
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import golois  # Librairie C++ fournie (jeu de données Go)
from tensorflow.keras import layers, regularizers
from tensorflow import keras
import gc

import json
import matplotlib.pyplot as plt
from google.colab import files



# Utilitaires internes (si tu veux factoriser ton code)
# from utils.model_blocks import dw_conv_block, se_block, attention_residual_block
# from utils.training_utils import moving_average, save_plots


In [4]:
# ================================================================
# ✅ GO-ALPHA HYBRID ~99k PARAMS — 60 EPOCHS — GOLOIS COMPAT
# ================================================================
import os, gc, shutil, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from datetime import datetime
import golois

# -----------------------------
# Runtime / GPU safety (Colab)
# -----------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU activé (memory growth).")
    except Exception as e:
        print("⚠️ GPU memory growth non appliqué:", e)
else:
    print("⚠️ Aucun GPU détecté — CPU utilisé.")

# ================================================================
# 🔧 PARAMS
# ================================================================
trainer = "AdamH"
base_title = "Model4B_AttentionHybrid"
date_str = datetime.now().strftime("%Y%m%d")

planes = 31            # 31 plans d'entrée (Go)
moves  = 361           # 19x19
N      = 10000         # taille buffer (prof)
epochs = 1000          # ← demandé
batch  = 128

# 🔥 capacité du modèle (≈99k params)
filters            = 110
se_reduction       = 10
value_dense_units  = 64

# Régularisation / Optim
l2_reg        = 1e-4
learning_rate = 1e-4
label_smooth  = 0.05

# ================================================================
# 📁 OUTPUT DIR
# ================================================================
model_name  = f"{date_str}_{trainer}_{base_title}_ep{epochs}_bs{batch}_lr{learning_rate}_l2{l2_reg}"
output_root = "output"
output_dir  = os.path.join(output_root, model_name)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)
print(f"🧾 Modèle : {model_name}")
print(f"📂 Dossier de sortie : {output_dir}")

# ================================================================
# 📊 DATA BUFFERS (mock init) — golois remplit ensuite
# ================================================================
input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy     = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value      = np.random.randint(2, size=(N,)).astype('float32')
end        = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')
groups     = np.zeros((N, 19, 19, 1), dtype='float32')

print("🔍 Test golois.getValidation()…")
golois.getValidation(input_data, policy, value, end)
print("✅ golois prêt.\n")

# ================================================================
# 🧠 MODEL — ATTENTION HYBRID (≈99k)
# ================================================================
def dw_conv_block(x, filters, l2_reg, stride=1):
    x = layers.SeparableConv2D(
        filters, kernel_size=3, strides=stride, padding='same', activation='relu',
        depthwise_regularizer=regularizers.l2(l2_reg),
        pointwise_regularizer=regularizers.l2(l2_reg),
        use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    return x

def se_block(x, reduction=10):
    c = int(x.shape[-1])
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(max(c // reduction, 1), activation='relu')(se)
    se = layers.Dense(c, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, c))(se)
    return layers.Multiply()([x, se])

def attention_residual_block(x, filters, l2_reg, reduction):
    shortcut = x
    x = dw_conv_block(x, filters, l2_reg)
    x = dw_conv_block(x, filters, l2_reg)
    x = se_block(x, reduction=reduction)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

inp = keras.Input(shape=(19, 19, planes), name='board')

# Stem
x = layers.Conv2D(filters, 1, activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(l2_reg),
                  use_bias=False)(inp)
x = layers.BatchNormalization()(x)

# 3 blocs résiduels avec SE + DW conv
for i in range(3):
    x = attention_residual_block(x, filters, l2_reg, reduction=se_reduction)
    if i % 2 == 1:
        x = layers.Dropout(0.10)(x)

# --- Policy Head ---
p = layers.Conv2D(1, 1, activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(l2_reg),
                  use_bias=False)(x)
p = layers.Flatten()(p)
policy_head = layers.Activation('softmax', name='policy')(p)

# --- Value Head ---
v = layers.GlobalAveragePooling2D()(x)
v = layers.Dense(value_dense_units, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg))(v)
v = layers.Dropout(0.15)(v)
value_head = layers.Dense(1, activation='sigmoid',
                          name='value',
                          kernel_regularizer=regularizers.l2(l2_reg))(v)

model = keras.Model(inputs=inp, outputs=[policy_head, value_head])
model.summary()

# Sanity check: ~99k params
total_params = model.count_params()
print(f"🔢 Total params: {total_params}")
assert 90_000 <= total_params < 110_000, f"❌ Total params hors cible: {total_params} (attendu ~99k)"

# ================================================================
# ⚙️ COMPILE + CALLBACKS
# ================================================================
optimizer    = keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
policy_loss  = keras.losses.CategoricalCrossentropy(label_smoothing=label_smooth)
value_loss   = keras.losses.MeanSquaredError()
metrics_dict = {'policy': ['categorical_accuracy'], 'value': ['mae']}

model.compile(optimizer=optimizer,
              loss={'policy': policy_loss, 'value': value_loss},
              loss_weights={'policy': 1.0, 'value': 1.0},
              metrics=metrics_dict)

ckpt_path   = os.path.join(output_dir, f"{model_name}_best.keras")
csv_path    = os.path.join(output_dir, f"{model_name}_training_log.csv")

# Ces callbacks ne seront actifs que sur les epochs avec validation (tous les 5)
common_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=ckpt_path, monitor='val_loss',
        save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2,
        min_lr=1e-6, verbose=1),                     # ↓ LR si val_loss stagne
    keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=6,
        restore_best_weights=True, verbose=1),
    keras.callbacks.CSVLogger(csv_path, append=True)
]

# Pour suivre (optionnel)
history_all = {
    "loss": [], "policy_categorical_accuracy": [], "policy_top5": [],
    "value_mae": [], "val_epochs": [],
    "val_loss": [], "val_policy_categorical_accuracy": [], "val_value_mae": []
}
top5_metric = keras.metrics.TopKCategoricalAccuracy(k=5, name="top5")

# ================================================================
# 🏋️‍♂️ TRAIN LOOP (style prof) + VALIDATION/CKPT tous les 5
# ================================================================
for i in range(1, epochs + 1):
    print(f"\n--- Époque {i}/{epochs} ---")

    # Remplissage buffer par golois
    golois.getBatch(input_data, policy, value, end, groups, i * N)

    # Val tous les 5 epochs (et dernière)
    do_val = (i % 5 == 0) or (i == epochs)
    validation_data = None
    callbacks = []
    if do_val:
        golois.getValidation(input_data, policy, value, end)
        validation_data = (input_data, [policy, value])
        callbacks = common_callbacks

    # Fit 1 epoch
    hist = model.fit(
        input_data, [policy, value],
        epochs=1, batch_size=batch, verbose=1,
        validation_data=validation_data,
        callbacks=callbacks
    )

    # Log metrics
    history_all["loss"].append(hist.history.get("loss", [None])[0])
    history_all["policy_categorical_accuracy"].append(
        hist.history.get("policy_categorical_accuracy", [None])[0]
    )
    # calc top5 manuellement (sur batch complet)
    preds_policy = model.predict(input_data, batch_size=batch, verbose=0)[0]
    history_all["policy_top5"].append(
        float(top5_metric(policy, preds_policy).numpy())
    )
    history_all["value_mae"].append(hist.history.get("value_mae", [None])[0])

    if do_val:
        history_all["val_epochs"].append(i)
        history_all["val_loss"].append(hist.history.get("val_loss", [None])[0])
        history_all["val_policy_categorical_accuracy"].append(
            hist.history.get("val_policy_categorical_accuracy", [None])[0]
        )
        history_all["val_value_mae"].append(hist.history.get("val_value_mae", [None])[0])

    # GC régulier
    if i % 5 == 0:
        gc.collect()
        print("🧹 GC.")

# Sauvegarde finale et résumé
final_path = os.path.join(output_dir, f"{model_name}_final.keras")
model.save(final_path)
print(f"\n💾 Sauvegardes :\n - Best: {ckpt_path}\n - Final: {final_path}")
print("✅ Entraînement terminé.")


✅ GPU activé (memory growth).
🧾 Modèle : 20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001
📂 Dossier de sortie : output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001
🔍 Test golois.getValidation()…
✅ golois prêt.



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board (InputLayer)  │ (None, 19, 19,    │          0 │ -                 │
│                     │ 31)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │      3,410 │ board[0][0]       │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        440 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 19, 19,    │     13,090 │ batch_normalizat… │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d… │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 19, 19,    │     13,090 │ batch_normalizat… │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d… │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 110)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 11)        │      1,221 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 110)       │      1,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 110) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 110)              │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 110)              │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 19, 19,    │          0 │ add[0][0]         │
│ (Activation)        │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 19, 19,    │     13,090 │ activation[0][0]  │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d… │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3  │ (None, 19, 19,    │     13,090 │ batch_normalizat… │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d

 Total params: 99,932 (390.36 KB)

 Trainable params: 98,392 (384.34 KB)

 Non-trainable params: 1,540 (6.02 KB)

🔢 Total params: 99932

--- Époque 1/1000 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 38s 188ms/step - loss: 6.1864 - policy_categorical_accuracy: 0.0048 - policy_loss: 5.9765 - value_loss: 0.1294 - value_mae: 0.3014

--- Époque 2/1000 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 5.6938 - policy_categorical_accuracy: 0.0261 - policy_loss: 5.4928 - value_loss: 0.1209 - value_mae: 0.2871

--- Époque 3/1000 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 5.3001 - policy_categorical_accuracy: 0.0557 - policy_loss: 5.0952 - value_loss: 0.1249 - value_mae: 0.2947

--- Époque 4/1000 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 4.9470 - policy_categorical_accuracy: 0.0981 - policy_loss: 4.7458 - value_loss: 0.1215 - value_mae: 0.2912

--- Époque 5/1000 ---
77/79 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 4.7888 - policy_categorical_accuracy: 0.1172 - policy_loss: 4.5888 - value_loss: 0.1205 - value_mae: 0.2898
Epoch 1: val_loss improved from inf to 5.89528, saving model to output/20251122_A

# Sauvegarde

In [5]:
# ================================================================
# 💾 SAUVEGARDE + 📊 VISUALISATION + 📈 LISSAGE + 📦 ZIP DOWNLOAD
# ================================================================

import json
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
import os
import shutil

# 🔹 Création du dossier (si manquant)
os.makedirs(output_dir, exist_ok=True)

# 🔹 Sauvegarde du modèle (.h5 & .keras)
h5_path = os.path.join(output_dir, f"{model_name}.h5")
keras_path = os.path.join(output_dir, f"{model_name}.keras")
hist_json = os.path.join(output_dir, "history.json")

model.save(h5_path)
model.save(keras_path)
print(f"✅ Modèles sauvegardés :\n - {h5_path}\n - {keras_path}")

# 🔹 Sauvegarde de l’historique complet
with open(hist_json, "w") as f:
    json.dump(history_all, f)
print(f"📘 Historique sauvegardé : {hist_json}")

# ================================================================
# 📊 VISUALISATION DES COURBES
# ================================================================
plt.style.use("seaborn-v0_8-whitegrid")

def save_plot(fig, name):
    fig.savefig(os.path.join(output_dir, name), dpi=300, bbox_inches='tight')

epochs_range = range(1, len(history_all.get("loss", [])) + 1)

# --- 1️⃣ Courbe de perte (Loss)
if history_all.get("loss"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["loss"], label='Training Loss', linewidth=2)
    if history_all.get("val_loss"):
        ax.plot(history_all["val_epochs"], history_all["val_loss"], label='Validation Loss', linewidth=2)
    ax.set_title(f"{model_name} - Loss")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Loss")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "loss_curve.png")
    plt.close(fig)

# --- 2️⃣ Policy Accuracy
if history_all.get("policy_categorical_accuracy"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_categorical_accuracy"], label='Policy Accuracy', linewidth=2)
    if history_all.get("val_policy_categorical_accuracy"):
        ax.plot(history_all["val_epochs"], history_all["val_policy_categorical_accuracy"], '--', label='Val Policy Acc', linewidth=2)
    ax.set_title(f"{model_name} - Policy Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "policy_accuracy.png")
    plt.close(fig)

# --- 3️⃣ Top-5 Accuracy
if history_all.get("policy_top5"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_top5"], label='Top-5 Accuracy', linewidth=2)
    if history_all.get("val_policy_top5"):  # ✅ clé optionnelle
        ax.plot(history_all["val_epochs"], history_all["val_policy_top5"], '--', label='Val Top-5', linewidth=2)
    ax.set_title(f"{model_name} - Top-5 Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "top5_accuracy.png")
    plt.close(fig)

# --- 4️⃣ Value MAE
if history_all.get("value_mae"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["value_mae"], label='Value MAE', linewidth=2)
    if history_all.get("val_value_mae"):
        ax.plot(history_all["val_epochs"], history_all["val_value_mae"], '--', label='Val Value MAE', linewidth=2)
    ax.set_title(f"{model_name} - Value MAE")
    ax.set_xlabel("Epochs"); ax.set_ylabel("MAE")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "value_mae.png")
    plt.close(fig)

# ================================================================
# 📈 LISSAGE (Policy Accuracy)
# ================================================================
def moving_average(data, window=5):
    data = np.array(data)
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

if history_all.get("policy_categorical_accuracy"):
    smooth_acc = moving_average(history_all["policy_categorical_accuracy"], 5)
    epochs_smooth = np.arange(5, len(history_all["policy_categorical_accuracy"]) + 1)
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(range(1, len(history_all["policy_categorical_accuracy"]) + 1),
            history_all["policy_categorical_accuracy"], alpha=0.4, label='Brut', linewidth=1.5)
    ax.plot(epochs_smooth, smooth_acc, color='red', linewidth=2.5,
            label='Moyenne glissante (5)')
    ax.set_title(f"{model_name} - Policy Accuracy (Lissée)")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.8)
    save_plot(fig, "policy_acc_smooth.png")
    plt.close(fig)

# ================================================================
# 🧾 EXPORT DU RÉSUMÉ DES MÉTRIQUES
# ================================================================
summary_path = os.path.join(output_dir, "metrics_summary.txt")
with open(summary_path, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Epochs: {len(history_all.get('loss', []))}\n\n")
    if history_all.get("loss"):
        f.write(f"Final Training Loss: {history_all['loss'][-1]:.4f}\n")
    if history_all.get("val_loss"):
        f.write(f"Final Validation Loss: {history_all['val_loss'][-1]:.4f}\n")
    if history_all.get("policy_categorical_accuracy"):
        f.write(f"Final Policy Accuracy: {history_all['policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("val_policy_categorical_accuracy"):
        f.write(f"Final Val Policy Accuracy: {history_all['val_policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("value_mae"):
        f.write(f"Final Value MAE: {history_all['value_mae'][-1]:.4f}\n")
    if history_all.get("val_value_mae"):
        f.write(f"Final Val Value MAE: {history_all['val_value_mae'][-1]:.4f}\n")
    if history_all.get("policy_top5"):
        f.write(f"Final Policy Top-5: {history_all['policy_top5'][-1]:.4f}\n")

print(f"\n✅ Fichiers et graphiques enregistrés dans : {output_dir}")

# ================================================================
# 📦 ZIP + TÉLÉCHARGEMENT AUTOMATIQUE
# ================================================================
zip_path = f"{output_dir}.zip"
shutil.make_archive(output_dir, 'zip', output_dir)
print(f"📦 Dossier compressé : {zip_path}")

try:
    files.download(zip_path)
    print(f"📥 Téléchargement du dossier complet lancé avec succès ✅")
except Exception as e:
    print(f"⚠️ Téléchargement automatique ignoré ({e})")


✅ Modèles sauvegardés :
 - output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001.h5
 - output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001.keras
📘 Historique sauvegardé : output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001/history.json

✅ Fichiers et graphiques enregistrés dans : output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001
📦 Dossier compressé : output/20251122_AdamH_Model4B_AttentionHybrid_ep1000_bs128_lr0.0001_l20.0001.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Téléchargement du dossier complet lancé avec succès ✅
